# Projet 3I026
##### Antoine Cadiou (3670631) & Vincent Jouve (3671083)

# Préparation du projet

Le projet est à rendre lors de la dernière séance de TD-TME de votre groupe qui aura lieu la semaine du **16 avril**. Lors de cette dernière séance, une soutenance sera organisée afin que vous présentiez le travail réalisé et les résultats obtenus.

## Données

Les données à utiliser sont des données issues d'une base de films et de notations faites par des spectateurs:
- base MovieLens : https://grouplens.org/datasets/movielens/
- base complémentaire `Movie Industrie`: https://www.kaggle.com/danielgrijalvas/movies


<font color="RED">**ATTENTION**: la base est très volumineuse (plus d'1Go) et ne pourra pas tenir sur votre répertoire de travail des machines de TME.</font>

Dans un premier temps, vous travaillerez sur la version réduite de cette base (`ml-latest-small.zip` sur le site movilens) que vous pouvez récupérer sur les machines de TD-TME: `/users/Enseignants/marsala/3i026-2019/MovieLens-small.tgz`
Cette version contient qu'une partie des lignes de la base originale.



En salle de TD-TME, vous accéderez aux données dans les fichiers suivants (à ne pas recopier !):
- répertoire : `/users/Enseignants/marsala/3i026-2019`
- répertoire des données MovieLens version réduite : `/users/Enseignants/marsala/3i026-2019/MovieLens-small/` 
- fichier archive avec les données MovieLesns version réduite : `/users/Enseignants/marsala/3i026-2019/MovieLens-small.tgz`
- répertoire des données MovieLens : `/users/Enseignants/marsala/3i026-2019/MovieLens/` 
  Il contient les fichiers CSV que vous pouvez ouvrir directement dans le notebook.
- fichier `/users/Enseignants/marsala/3i026-2019/kaggle-movies.csv` : infos sur les films de la base Movie Industrie
   
   
<font color="RED" size="+1">**[Q]**</font> Réaliser des expérimentations avec la base MovieLens version réduite.

Charger les données dans un dataframe, afficher des informations statistiques sur les données, commencer à réfléchir comment appliquer les classifieurs vus dans les séances précédentes sur ces données...

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# La ligne suivante permet de préciser le chemin d'accès à la librairie iads
import sys
sys.path.append('../')

# Importation de la librairie iads
import iads as iads

# importation de LabeledSet
from iads import LabeledSet as ls

# importation de Classifiers
from iads import Classifiers as cl

# importation de utils
from iads import utils_iads as ut

import random

In [2]:
def accuracy(LSet,Class):
    somme=0
    for i in range(LSet.size()):
        somme += (Class.predict(LSet.getX(i))-LSet.getY(i)[0])**2
    return somme/LSet.size()

In [3]:
links = pd.read_csv("data/links.csv")
movies = pd.read_csv("data/movies.csv")
movies_kaggle = pd.read_csv("data/movies_kaggle.csv", encoding = "cp1252")
ratings = pd.read_csv("data/ratings.csv", parse_dates=[-1])
#tags = pd.read_csv("data/tags.csv")

In [4]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies_kaggle.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<br><br><br><br><br><br>
<font color="green"> Nous allons essayer dans un premier temps de prédire le score d'un film en fonction de son budget et de son genre</font>

In [9]:
genres = np.unique(movies_kaggle['genre'])
genres

array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Drama', 'Family', 'Fantasy', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], dtype=object)

<font color="red">Nous récupérons les colonnes qui nous interesse dans la base de données,<br>
Nous créons autant de colonne que de genre avec des 0 ou des 1 si le film appartient à tel ou tel genre,<br>
Nous créons des colonnes pour les attributs qui ont besoins d'être normalisés</font>

In [10]:
#on garde les colonnes que l'on veut traiter
tmp = movies_kaggle.loc[:,["score", "genre", "budget", "runtime"]]
#on supprime les lignes où le budget = 0
tmp = tmp[tmp.budget > 0]

#on crée des attributs(colonnes) et on normalise
#tmp['budgetPerMin'] = tmp['budget']/tmp['runtime']

for g in genres:
    tmp[g] = (tmp['genre']==g).astype(int)

    

moyenne_budget = tmp['budget'].mean()
ecarttype_budget = tmp['budget'].std()
tmp['budgetNormalise'] = (tmp['budget']-moyenne_budget)/ecarttype_budget

moyenne_runtime = tmp['runtime'].mean()
ecarttype_runtime = tmp['runtime'].std()
tmp['runtimeNormalise'] = (tmp['runtime']-moyenne_runtime)/ecarttype_runtime 

"""
moyenne_perMin = tmp['budgetPerMin'].mean()
ecarttype_perMin = tmp['budgetPerMin'].std()
tmp['budgetPerMinNormalise'] = (tmp['budgetPerMin']-moyenne_perMin)/ecarttype_perMin 
"""

#le nombre de dimension de notre problème
dimension = len(genres)+2
print("nb_dimension =", dimension)


#On initialise le labelset
the_set = ls.LabeledSet(dimension)
for row in tmp.itertuples():
    the_set.addExample(row[5:], row[1])

#on le split
train, test = the_set.split(0.8)
print(the_set.size(), train.size(), test.size())

tmp.head()

nb_dimension = 19
4638 3711 927


,score,genre,budget,runtime,Action,Adventure,Animation,Biography,Comedy,Crime,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,budgetNormalise,runtimeNormalise
0,8.1,Adventure,8000000.0,89,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.704177,-1.031778
1,7.8,Comedy,6000000.0,103,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,-0.754216,-0.254984
2,6.9,Action,15000000.0,110,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.529044,0.133414
3,8.4,Action,18500000.0,137,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.441477,1.631517
4,6.9,Adventure,9000000.0,90,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.679158,-0.976292


In [11]:
the_set.getX(0)

array([ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.70417745, -1.0317777 ])

#### On test avec un Classifier sur la moyenne (solution naïve)

In [12]:
class ClassifierMoyenne(cl.Classifier):
    """ Classe pour représenter un classifieur par K plus proches voisins.
        Cette classe hérite de la classe Classifier
    """

    def __init__(self):
        """ Constructeur de Classifier
            Argument:
                - intput_dimension (int) : dimension d'entrée des exemples
            Hypothèse : input_dimension > 0
        """
        self.moyenne = 0

    def predict(self, x):
        return self.moyenne


    def train(self, labeledSet):
        """ Permet d'entrainer le modele sur l'ensemble donné
        """
        for i in range(labeledSet.size()):
            self.moyenne += labeledSet.getY(i)[0]
        self.moyenne /= labeledSet.size()

In [13]:
classifier3 = ClassifierMoyenne()
classifier3.train(train)

acc = 0
for i in range(test.size()):
    if (i<10):
        print(test.getY(i)[0], "|", classifier3.predict(test.getX(i)))
    acc += abs(test.getY(i)[0] - classifier3.predict(test.getX(i)))
print(acc/test.size())
print(accuracy(test, classifier3))

5.4 | 6.330396119644291
4.9 | 6.330396119644291
6.9 | 6.330396119644291
5.8 | 6.330396119644291
6.1 | 6.330396119644291
5.8 | 6.330396119644291
6.3 | 6.330396119644291
6.2 | 6.330396119644291
5.4 | 6.330396119644291
7.3 | 6.330396119644291
0.709628943602461
0.8182037856747324


#### On test avec un Classifier Gradient Batch (qui pourrait permettre de gérer autant de dimension)

In [14]:
classifier = cl.ClassifierGradientBatch(dimension, 0.001)
print(train.size())
for i in range(10):
    classifier.train(train)
print(classifier.w)

3711
[ 1.59914496e+03  2.98007361e+02 -2.43533961e+02  4.77447799e+02
 -3.35142380e+03  5.75378390e+01  7.06737082e+02 -2.45226748e+01
 -8.05271823e+01 -7.09540126e+02 -6.60115499e+00  2.88620941e+01
 -2.44031058e+00 -1.04839358e+01 -1.73137919e+01  0.00000000e+00
 -9.83989996e-01  1.26514645e+04  1.86584626e+04]


In [15]:
acc = 0
for i in range(test.size()):
    if (i<10):
        print(test.getY(i)[0], "|", classifier.predict(test.getX(i)))
    acc += abs(test.getY(i)[0] - classifier.predict(test.getX(i)))
print(acc/test.size())
print(accuracy(test,classifier))

5.4 | -6090.856939118827
4.9 | -13533.325244227466
6.9 | -6569.705377994345
5.8 | -17672.11259740028
6.1 | -10113.751311065233
5.8 | 560.5397267874539
6.3 | -15375.788891867624
6.2 | 10175.326405664913
5.4 | 2262.1220122181844
7.3 | 5355.83325560407
21412.029595907396
854756964.6574665


On prédit une note à + ou - 2.85 points/10, c'est très mauvais quand on sait que le classifieur naïf fait du + ou - 0,71

#### On test avec un Classifier KNN

In [16]:
class ClassifierKNN(cl.Classifier):
    """ Classe pour représenter un classifieur par K plus proches voisins.
        Cette classe hérite de la classe Classifier
    """

    def __init__(self, input_dimension, k):
        """ Constructeur de Classifier
            Argument:
                - intput_dimension (int) : dimension d'entrée des exemples
            Hypothèse : input_dimension > 0
        """
        self.k = k
        self.dim=input_dimension

    def predict(self, x):
        """ rend la prediction sur x (-1 ou +1)
        """
        def dist(u,v):
            dist = (u-v)**2
            dist = sum(dist)
            return dist

        d = np.array([dist(x,self.labeledSet.getX(i)) for i in range(self.labeledSet.size())])
        dSort=np.argsort(d)
        value = 0
        
        for i in range(self.k):
            value += self.labeledSet.getY(dSort[i])

        return value/self.k


    def train(self, labeledSet):
        """ Permet d'entrainer le modele sur l'ensemble donné
        """
        self.labeledSet = labeledSet

In [17]:
classifier2 = ClassifierKNN(dimension, 3)
classifier2.train(train)

In [18]:
acc = 0
for i in range(test.size()):
    if (i<10):
        print(test.getY(i)[0], "|", classifier2.predict(test.getX(i))[0])
    acc += abs(test.getY(i)[0] - classifier2.predict(test.getX(i))[0])
print(acc/test.size())
print(accuracy(test,classifier2))

5.4 | 5.966666666666666
4.9 | 6.266666666666667
6.9 | 6.2
5.8 | 6.666666666666667
6.1 | 6.533333333333332
5.8 | 6.066666666666666
6.3 | 5.8999999999999995
6.2 | 6.3999999999999995
5.4 | 5.833333333333333
7.3 | 7.266666666666667
0.6893923049262866
[0.80054417]


On prédit une note à + ou - 0.64 points/10, ce qui est meilleur que le classifieur naïf

In [52]:
stars, count = np.unique(movies_kaggle['star'],return_counts=True)
starstab=[]
ind=np.argsort(count)
for i in range(len(ind)):
    if(count[ind[i]]>10):
        starstab.append(stars[ind[i]])
#dict(zip(stars, count))


In [53]:
#on garde les colonnes que l'on veut traiter
tmp2 = movies_kaggle.loc[:,["gross", "genre", "budget", "runtime", "star"]]
#on supprime les lignes où le budget = 0
tmp2 = tmp2[tmp2.budget > 0]

#on crée des attributs(colonnes) et on normalise
for g in genres:
    tmp2[g] = (tmp2['genre']==g).astype(int)

for s in starstab:
    tmp2[s] = (tmp2['star']==s).astype(int)

tmp2['starOther'] = 0
    
for i in range(len(tmp2)):
    tmp2.iloc[i,-1] = (1 if (tmp2.iloc[i,4] not in starstab) else 0)

moyenne_budget = tmp2['budget'].mean()
ecarttype_budget = tmp2['budget'].std()
tmp2['budgetNormalise'] = (tmp2['budget']-moyenne_budget)/ecarttype_budget

moyenne_runtime = tmp2['runtime'].mean()
ecarttype_runtime = tmp2['runtime'].std()
tmp2['runtimeNormalise'] = (tmp2['runtime']-moyenne_runtime)/ecarttype_runtime 


del tmp2["genre"]
del tmp2["budget"]
del tmp2["runtime"]
del tmp2["star"]
#le nombre de dimension de notre problème
dimension = len(tmp2.loc[0])-1
print("nb_dimension =", dimension)


#On initialise le labelset
the_set2 = ls.LabeledSet(dimension)
for row in tmp2.itertuples():
    the_set2.addExample(row[2:], row[1])

#on le split
train2, test2 = the_set2.split(0.80)
print(the_set2.size(), train2.size(), test2.size())
tmp2.head()

nb_dimension = 140
4638 3711 927


,gross,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Adam Sandler,Johnny Depp,Bruce Willis,Tom Hanks,Denzel Washington,Robert De Niro,Nicolas Cage,starOther,budgetNormalise,runtimeNormalise
0,52287414.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,-0.704177,-1.031778
1,70136369.0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.754216,-0.254984
2,179800601.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.529044,0.133414
3,85160248.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.441477,1.631517
4,18564613.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,-0.679158,-0.976292


In [54]:
classifier4 = ClassifierKNN(dimension, 1)
classifier4.train(train2)

acc = 0
for i in range(test2.size()):
    if (i<10):
        print(test2.getY(i)[0], "|", classifier4.predict(test2.getX(i))[0])
    acc += abs(test2.getY(i)[0] - classifier4.predict(test2.getX(i))[0])
print(acc/test2.size())
print(accuracy(test2,classifier4))

37662162.0 | 24042490.0
37300107.0 | 3525161.0
58709717.0 | 30336800.0
18942396.0 | 1579260.0
14008193.0 | 57387516.0
10324441.0 | 15156200.0
6923891.0 | 2708188.0
78046570.0 | 33105600.0
10706786.0 | 15279680.0
46412041.0 | 21706100.0
45494453.850053936
[5.88185132e+15]
